In [ ]:
# Notebook: CNN Deep Learning Model

import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [202]:
X_train = pd.read_csv('../data/processed/X_train_scaled.csv')
X_test = pd.read_csv('../data/processed/X_test_scaled.csv')
Y_train = pd.read_csv('../data/processed/y_train.csv')
Y_test = pd.read_csv('../data/processed/y_test.csv')

Y_train

,label
0,3
1,6
2,2
3,2
4,13
...,...
27450,13
27451,23
27452,18
27453,17


In [203]:
# Reshape images to (28, 28, 1) for modeling

X_train = X_train.to_numpy().reshape(-1, 28, 28, 1)
X_test = X_test.to_numpy().reshape(-1, 28, 28, 1)

In [204]:
# Fix encoding for categorical_crossentropy

Y_train = to_categorical(Y_train, num_classes=25)
Y_test = to_categorical(Y_test, num_classes=25)

print(Y_train.shape)
print(Y_test.shape)

(27455, 25)
(7172, 25)


In [205]:
model = Sequential()

In [206]:
# Adding the first convolutional and pooling layer

model.add(Conv2D(128, kernel_size=(5,5), strides=1, padding='same', 
                    activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(3,3), strides=2, padding='same'))

/Users/adityasharma/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [207]:
# Adding the second convolutional and pooling layer

model.add(Conv2D(64, kernel_size=(2,2), strides=1, activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2,2), strides=2, padding='same'))

In [208]:
# Adding the third convolutional and pooling layer

model.add(Conv2D(32, kernel_size=(2,2), strides=1, activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2,2), strides=2, padding='same'))

In [209]:
model.add(Flatten())  # turn 3D into 1D (rolling image into list)

In [210]:
model.add(Dense(512, activation='relu'))  # fully connected layer to learn combinations of features
model.add(Dropout(0.25))  # prevent overfitting by turning of 25% of neurons
model.add(Dense(25, activation='softmax'))  # final output layer. classifies 25 letters.
model.summary()

Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_49 (Conv2D)              │ (None, 28, 28, 128)    │         3,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_47 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_50 (Conv2D)              │ (None, 14, 14, 64)     │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_48 (MaxPooling2D) │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_51 (Conv2D)              │ (None, 7, 7, 32)       │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_49 (MaxPooling2D) │ (None, 4, 4, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 25)             │        12,825 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 319,865 (1.22 MB)

 Trainable params: 319,865 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [211]:
# Compile model using automatic learning rate adjustment for multi-class classification

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [212]:
train_datagen = ImageDataGenerator(
    rotation_range=0,
    height_shift_range=0.2,
    width_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

model.fit(
    train_datagen.flow(X_train, Y_train, batch_size=200),
    epochs=35,
    validation_data=(X_test, Y_test),
    shuffle=1
)


Epoch 1/35


/Users/adityasharma/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 58ms/step - accuracy: 0.0641 - loss: 3.1408 - val_accuracy: 0.2648 - val_loss: 2.3800
Epoch 2/35
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 59ms/step - accuracy: 0.2307 - loss: 2.4859 - val_accuracy: 0.5612 - val_loss: 1.4205
Epoch 3/35
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 58ms/step - accuracy: 0.4156 - loss: 1.7937 - val_accuracy: 0.6036 - val_loss: 1.1814
Epoch 4/35
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 59ms/step - accuracy: 0.5395 - loss: 1.3666 - val_accuracy: 0.6903 - val_loss: 0.8487
Epoch 5/35
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 59ms/step - accuracy: 0.6295 - loss: 1.1026 - val_accuracy: 0.7310 - val_loss: 0.7336
Epoch 6/35
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.6783 - loss: 0.9413 - val_accuracy: 0.7638 - val_loss: 0.6240
Epoch 7/35
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 60ms/step - accuracy: 0.7196 - loss: 0.8144 - val_accuracy: 0.8024 - val_loss: 0.5311
Epoch 8/35
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.7587 - loss: 0.7149 - val_accuracy: 0.853

In [216]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test accuracy: {accuracy * 100}%")
print(f"Final loss: {loss}")

225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9928 - loss: 0.0206
Test accuracy: 99.4004487991333%
Final loss: 0.01947113126516342


In [218]:
model.save("../models/asl_model.keras")